In [19]:
from bs4 import BeautifulSoup
'''Доступ к Uniprot'''
from Bio import ExPASy
from Bio import SwissProt
import pandas as pd

columns = []
d = []
with open('With_Cys/pfam_1.txt', 'r') as f:
    for line in f:
        if line.startswith('<'):
            columns = line.split('> <')
        else:
            d.append(line.split())
columns[0] = columns[0].strip('<')
columns[-1] = columns[-1].strip('>\n')

data = pd.DataFrame(d, columns=columns)

with open("With_Cys/Cys_sequences_1.xml","r") as f:
    contents = f.read()
soup = BeautifulSoup(contents,'xml')

sequences = soup.sequences.findAll('sequence')

In [33]:
seqs = {}
for seq in sequences:
    for seg in seq.find_all('seg'):
        if bool(seg.find(idx = '0')) == True and bool(seg.find(idx = '1')) == True:
            hits = seg.find_all('hit')
            if float(hits[0]['pvalue']) <= 1e-15 and float(hits[1]['pvalue']) <= 1e-15:
                seqs.setdefault(seq['name'], []).append([int(hits[0]['pos']), int(hits[1]['pos'])])

In [7]:
with open('With_Cys/final_proteins_1.fasta', 'r') as f:
    fasta = {}
    for line in map(lambda st: st.rstrip('\n'), f):
        if line.startswith('>'):
            name = line.lstrip('>')
        else:
            fasta.setdefault(name, []).append(line)

In [43]:
new_seq = {}
for key in seqs.keys():
    if key in fasta:
        new_seq[key] = seqs[key]
seqs = new_seq

In [46]:
data = data[data['seq id'].isin(seqs.keys())].reset_index(drop = True)

In [55]:
data

,seq id,alignment start,alignment end,envelope start,envelope end,hmm acc,hmm name,type,hmm start,hmm end,hmm length,bit score,E-value,significance,clan
0,A0A0J6VGA8,62,156,52,157,PF13426.7,PAS_9,Domain,9,103,104,69.7,2.1e-19,1,CL0183
1,A0A0J6VGA8,175,235,173,237,PF00512.25,HisKA,Domain,3,64,67,34.0,2.2e-08,1,CL0025
2,A0A0J6VGA8,280,402,279,405,PF02518.26,HATPase_c,Domain,2,109,112,49.2,6.5e-13,1,CL0025
3,A0A0J6VGA8,427,537,427,538,PF00072.24,Response_reg,Domain,1,111,112,67.4,1.1e-18,1,CL0304
4,A0A059UGE7,331,427,324,428,PF13426.7,PAS_9,Domain,7,103,104,69.9,1.9e-19,1,CL0183
5,A0A059UGE7,633,725,624,728,PF13426.7,PAS_9,Domain,9,101,104,65.0,6.1e-18,1,CL0183
6,A0A059UGE7,813,1099,811,1099,PF00069.25,Pkinase,Domain,3,264,264,204.9,1.4e-60,1,CL0016
7,A0A098T9D3,47,149,46,150,PF13426.7,PAS_9,Domain,2,103,104,61.8,6.1e-17,1,CL0183
8,A0A098T9D3,162,308,162,311,PF00990.21,GGDEF,Domain,1,156,161,60.0,2.2e-16,1,CL0276
9,A0A098T9D3,333,568,332,568,PF00563.20,EAL,Domain,2,238,238,230.3,2.3e-68,1,CL0276


In [90]:
def IfLOV(row):
    b = False
    if row['clan'] == 'CL0183':
        for hit in seqs[row['seq id']]:
            if (int(row[3]) - hit[0]) <=5 and (hit[1] + 43 - int(row[4])) <= 1:
                return 'LOV'
    return row['hmm name']
        

In [89]:
seqs['A0A059UGE7']

[[321, 366], [621, 666]]

In [93]:
data.apply(IfLOV, axis = 1)

0                 LOV
1               HisKA
2           HATPase_c
3        Response_reg
4                 LOV
5                 LOV
6             Pkinase
7                 LOV
8               GGDEF
9                 EAL
10                LOV
11                LOV
12            Pkinase
13                LOV
14                LOV
15            Pkinase
16                LOV
17              GGDEF
18                EAL
19                LOV
20            Kelch_4
21            Kelch_3
22            Kelch_3
23            Kelch_4
24                LOV
25              F-box
26            Kelch_4
27            Kelch_3
28            Kelch_4
29                GAF
             ...     
17270         Kelch_3
17271         Kelch_3
17272             LOV
17273           PAS_9
17274         Pkinase
17275             LOV
17276      F-box-like
17277         Kelch_4
17278         Kelch_3
17279         Kelch_4
17280             LOV
17281           PAS_9
17282         Pkinase
17283             LOV
17284     

In [ ]:
for k in seqs.keys():
    seq_data = data.loc[data['seq id'] == k]
    df = seq_data.loc[seq_data['clan'] == 'CL0183']
    b = False
    for hit in seqs[k]:
        for i in range(df.shape[0]):
            if int(df.iloc[i, 3]) - hit[0] <=5 and hit[1] + 43 - int(df.iloc[i, 4]) <= 1:
                b = True
    if not b:
        data.drop(data[data['seq id'] == k].index, inplace=True)
        wrong.append(k)
    else:
        new_names.append(k)